In [ ]:
from pathlib import Path
import os
if not str(Path.cwd()).endswith('/Cloud-Telescope'):
    os.chdir(Path.cwd().parent)
    print(f'New working dir is {Path.cwd()}')    

import importlib
import pandas as pd

import src.d01_data.read_files
importlib.reload(src.d01_data.read_files)

import src.d01_data.save_files
importlib.reload(src.d01_data.save_files)

import src.d01_data.log_names_enum
importlib.reload(src.d01_data.log_names_enum)

import src.d02_intermediate.add_colums_to_conn
importlib.reload(src.d02_intermediate.add_colums_to_conn)

import src.d03_visiualisation.tabel_visiualisation
importlib.reload(src.d03_visiualisation.tabel_visiualisation)

import src.d02_intermediate.merge_pcap_files
importlib.reload(src.d02_intermediate.merge_pcap_files)

import src.d02_intermediate.script_merge_descriptor
importlib.reload(src.d02_intermediate.script_merge_descriptor)

import src.config
importlib.reload(src.config)

### Import .log Files as Dataframes and save them as feather files

In [ ]:
from src.d01_data.read_files import *
from src.d01_data.log_names_enum import LogNamesEnum
from src.d01_data.save_files import *
from src.d02_intermediate.add_colums_to_conn import *

path_to_log: str = 'cloud_merged_pcap/zeek_final'

def import_and_save_all(description: str):
    for log_name in LogNamesEnum:
        print(f'reading {log_name} ...')
        dataframe: pd.DataFrame = read_log_file(log_name, path_to_log, False)
        print(f'saving {log_name} ...')
        save_dataframe_as_feather(dataframe, log_name, description)
        print(f'finished {log_name}')

def import_and_save_specific(log_name: LogNamesEnum):
    print(f'reading {log_name} ...')
    dataframe: pd.DataFrame = read_log_file(log_name, path_to_log, False)
    print(f'saving {log_name} ...')
    save_dataframe_as_feather(dataframe, log_name, 'init')
    print(f'finished {log_name}')

#import_and_save_all('init-cloud')
#import_and_save_specific(LogNamesEnum.dns)
#import_and_save_specific(LogNamesEnum.notice)
import_and_save_specific(LogNamesEnum.notice_ws_ddos)

### Convert json Geo-Databases to csv files

In [ ]:
from src.d02_intermediate.merge_pcap_files import *

def convert_json_to_csv_notebook():
    convert_json_to_csv('/ipinfo/database')

# convert_json_to_csv_notebook()

### save conn orgin ips als ip list

In [ ]:
def save_conn_ip():
    conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
    orig_ip: pd.DataFrame = conn['id.orig_h']
    orig_ip.to_csv(f'{path_root}/Cloud-Telescope/data/raw/merge_pcap_test/ips_cloud.txt', index=False, header=False)
def save_conn_resp_ip():
    conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
    orig_ip: pd.DataFrame = conn['id.resp_h']
    orig_ip.to_csv(f'{path_root}/Cloud-Telescope/data/raw/merge_pcap_test/ips_cloud_resp.txt', index=False, header=False)  
#save_conn_ip()
save_conn_resp_ip()

### Merge IpInfo (IP-GEO csv files) into conn 

In [ ]:
from src.d02_intermediate.add_colums_to_conn import *
from src.d01_data.read_files import *
from src.d01_data.save_files import *
from src.config import *
import pandas as pd

def test_loading_csv():
    csv: pd.DataFrame = pd.read_csv(f'{path_root}/Cloud-Telescope/data/ipInfo/2025-01-27/2025-01-27_countryasn_merge.csv')
    csv2: pd.DataFrame = csv[csv['ip'] == '152.32.237.160']
    return csv2

def merge_ip_info_notebook(ip_info_merge_folder: str) -> pd.DataFrame:
    conn_df: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
    ip_info_merge_folder_path: str = f'ipInfo/{ip_info_merge_folder}'
    conn_ip_infos: pd.DataFrame = merge_ip_info(conn_df, ip_info_merge_folder_path)
    save_dataframe_as_feather(conn_ip_infos, LogNamesEnum.conn, 'ipInfo')
    return conn_ip_infos

merge_ip_info_notebook('2025-01-27')

### merge descriptor files

In [ ]:
from src.d02_intermediate.script_merge_descriptor import extract_data_from_descriptors
from src.d02_intermediate.merge_pcap_files import *
from src.config import *
from src.d01_data.log_names_enum import *
from src.d01_data.save_files import *
from src.d01_data.read_files import *

def merge_descriptor_files():
    extract_data_from_descriptors('/Cloud-Telescope/data/raw/cloud_merged_pcap/descriptor', '/Cloud-Telescope/data/raw/cloud_merged_pcap')

def merge_descriptor_conn():
    conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)    
    
    descriptors: pd.DataFrame = pd.read_csv(f'{path_root}/Cloud-Telescope/data/raw/cloud_merged_pcap/descriptor.csv')
    
    conn['creation'] = pd.NA
    conn['deletion'] = pd.NA
    conn['region'] = pd.NA
    
    # for index, row in conn.iterrows():
    #     if row['history'] != '^d':
    #         for index_des, row_des in descriptors.iterrows():
    #             if row['id.orig_h'] == row_des['ipv4']:
    #                 conn.loc[index, 'creation'] = row_des['creation'] 
    #                 conn.loc[index, 'deletion'] = row_des['deletion']   
    #                 conn.loc[index, 'region'] = row_des['region']
    #                 break
    #     if row['history'] == '^d':
    #         for index_des, row_des in descriptors.iterrows():
    #             if row['id.resp_h'] == row_des['ipv4']:
    #                 conn.loc[index, 'creation'] = row_des['creation'] 
    #                 conn.loc[index, 'deletion'] = row_des['deletion']   
    #                 conn.loc[index, 'region'] = row_des['region']
    #                 break
    conn = conn.merge(descriptors, left_on='id.orig_h', right_on='ipv4', how='left', suffixes=('', '_desc'))
    
    mask = conn['history'] == '^d'
    conn.loc[mask, ['creation', 'deletion', 'region']] = conn.loc[mask, ['creation_desc', 'deletion_desc', 'region_desc']]

    conn = conn.drop(columns=['ipv4', 'creation_desc', 'deletion_desc', 'region_desc'])
    
    save_dataframe_as_feather(conn, LogNamesEnum.conn, 'descriptor')    
    
    

# convert_json_to_csv_notebook()
merge_descriptor_conn()

### Add country Name column to conn.log

In [ ]:
from src.d02_intermediate.add_colums_to_conn import add_country_name
from src.d02_intermediate.switch_two_colums import switch_two_columns

def add_country_name_to_conn():
    raw_conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
    conn_country_name: pd.DataFrame = add_country_name(raw_conn)
    conn_country_name = switch_two_columns(conn_country_name, 'orig_country_name', 'geo.orig.country_code')
    conn_country_name = switch_two_columns(conn_country_name, 'resp_country_name', 'geo.resp.country_code')
    save_dataframe_as_feather(conn_country_name, LogNamesEnum.conn, 'CountryName')
#add_country_name_to_conn()

### Add date column to conn

In [ ]:
def add_data_column():
    conn: pd.DataFrame = load_latest_dataframe(LogNamesEnum.conn)
    conn = add_column_data(conn)
    conn = switch_two_columns(conn, 'ts', 'Date')
    save_dataframe_as_feather(conn, LogNamesEnum.conn, 'Date')
    
#add_data_column()

### Test mergecap tool merging .pcap files 

In [ ]:
from src.config import *
import pandas as pd
import numpy as np
from src.d01_data.read_files import *
import os

def test_mergecap():
    conn: pd.DataFrame = read_log_file(LogNamesEnum.conn, 'cloud_merged_pcap/zeek_v1', False)
    #ips: list[str] = os.listdir(f'{path_server_data}/run200125') 
    #ips.remove('empty.txt')
    #ips.remove('test.sh')
    ips = np.array(['104-131-10-181', '104-131-159-110', '104-131-159-155', '104-131-180-133', '104-131-182-217', '104-131-184-237', '104-131-189-13', '104-131-190-169', '104-131-193-189', '104-131-34-32', '104-131-87-89', '104-236-114-0', '104-236-135-143', '104-236-235-1', '104-236-6-168', '104-248-131-80', '104-248-137-164', '104-248-152-220', '104-248-157-116', '104-248-158-194', '104-248-17-211', '104-248-177-8', '104-248-178-205', '104-248-187-110', '104-248-187-210', '104-248-221-97', '104-248-29-161', '104-248-54-237', '104-248-76-56', '104-248-88-57', '104-248-94-211', '107-170-0-51', '107-170-11-246', '107-170-14-105', '107-170-14-217', '107-170-15-103', '107-170-19-102', '107-170-197-65', '107-170-197-66', '107-170-226-90', '107-170-227-221', '107-170-230-244', '107-170-236-219', '107-170-242-33', '107-170-242-59', '107-170-244-162', '107-170-245-36', '107-170-249-206', '107-170-255-222', '107-170-34-87', '107-170-56-114', '107-170-56-161', '107-170-57-43', '107-170-63-238', '107-170-7-204', '107-170-71-69', '107-170-71-93', '107-170-77-148', '128-199-138-38', '128-199-149-59', '128-199-16-110', '128-199-16-127', '128-199-225-29', '128-199-249-47', '128-199-251-103', '128-199-52-222', '134-122-25-99', '134-122-36-48', '134-122-39-189', '134-122-81-143', '134-122-88-82', '134-122-94-44', '134-209-118-177', '134-209-153-153', '134-209-154-179', '134-209-161-200', '134-209-173-156', '134-209-186-80', '134-209-190-143', '134-209-206-198', '134-209-39-168', '134-209-67-74', '134-209-86-248', '134-209-97-241', '137-184-107-62', '137-184-124-183', '137-184-15-176', '137-184-150-21', '137-184-153-197', '137-184-163-216', '137-184-165-5', '137-184-171-112', '137-184-171-90', '137-184-175-103', '137-184-175-247', '137-184-206-7', '137-184-237-204', '137-184-25-143', '137-184-254-167', '137-184-255-29', '137-184-255-43', '137-184-5-187', '137-184-54-96', '137-184-69-137', '137-184-69-176', '137-184-84-116', '137-184-87-193', '137-184-95-156', '138-197-101-24', '138-197-122-97', '138-197-129-173', '138-197-131-189', '138-197-132-155', '138-197-133-243', '138-197-134-40', '138-197-136-241', '138-197-136-73', '138-197-137-27', '138-197-140-151', '138-197-147-12', '138-197-149-167', '138-197-149-19', '138-197-161-75', '138-197-167-183', '138-197-167-229', '138-197-18-203', '138-197-201-63', '138-197-205-139', '138-197-215-178', '138-197-24-20', '138-197-29-168', '138-197-71-166', '138-197-79-1', '138-197-83-147', '138-197-90-14', '138-197-91-67', '138-68-1-216', '138-68-1-97', '138-68-11-104', '138-68-128-162', '138-68-147-58', '138-68-154-180', '138-68-157-95', '138-68-161-118', '138-68-168-76', '138-68-179-237', '138-68-185-133', '138-68-187-105', '138-68-23-103', '138-68-231-172', '138-68-232-124', '138-68-235-125', '138-68-235-245', '138-68-241-76', '138-68-245-227', '138-68-29-162', '138-68-29-195', '138-68-4-160', '138-68-50-121', '138-68-51-84', '138-68-56-233', '138-68-6-200', '138-68-72-15', '138-68-9-115', '138-68-92-214', '139-59-105-98', '139-59-114-160', '139-59-115-99', '139-59-121-18', '139-59-125-194', '139-59-134-88', '139-59-135-54', '139-59-152-190', '139-59-167-55', '139-59-17-59', '139-59-18-123', '139-59-2-169', '139-59-2-234', '139-59-20-159', '139-59-23-111', '139-59-230-1', '139-59-230-8', '139-59-235-101', '139-59-241-133', '139-59-246-45', '139-59-30-170', '139-59-30-215', '139-59-38-176', '139-59-6-131', '139-59-6-192', '139-59-7-179', '139-59-72-55', '139-59-76-219', '139-59-82-127', '139-59-83-126', '139-59-87-161', '139-59-9-234', '139-59-9-236', '139-59-9-51', '142-93-0-175', '142-93-111-243', '142-93-121-93', '142-93-132-216', '142-93-156-172', '142-93-177-13', '142-93-18-153', '142-93-18-170', '142-93-196-39', '142-93-210-126', '142-93-22-107', '142-93-22-62', '142-93-223-48', '142-93-27-134', '142-93-27-77', '142-93-32-213', '142-93-35-81', '142-93-46-121', '142-93-71-167', '142-93-84-189', '142-93-84-3', '142-93-87-49', '142-93-9-201', '143-110-135-199', '143-110-135-220', '143-110-139-32', '143-110-142-6', '143-110-154-204', '143-110-176-189', '143-110-187-84', '143-110-192-58', '143-110-194-221', '143-110-203-141', '143-110-206-211', '143-110-209-100', '143-110-212-102', '143-110-213-87', '143-110-232-159', '143-198-154-15', '143-198-184-205', '143-198-218-183', '143-198-223-155', '143-198-236-133', '143-198-33-181', '143-198-37-10', '143-198-37-227', '143-198-65-128', '143-198-65-135', '143-198-72-151', '143-198-9-116', '143-198-94-34', '143-198-98-182', '143-244-139-232', '143-244-140-73', '143-244-187-150', '144-126-205-241', '144-126-209-67', '144-126-212-8', '144-126-214-173', '144-126-218-173', '144-126-235-189', '144-126-239-137', '146-185-129-240', '146-185-132-179', '146-185-135-212', '146-185-139-167', '146-185-148-74', '146-185-153-227', '146-185-153-235', '146-185-158-167', '146-185-159-128', '146-185-162-153', '146-185-162-191', '146-185-165-68', '146-185-167-49', '146-185-171-207', '146-185-171-209', '146-185-173-165', '146-185-175-243', '146-185-176-80', '146-185-176-97', '146-185-177-162', '146-185-178-228', '146-185-182-168', '146-185-183-82', '146-185-187-37', '146-190-125-141', '146-190-131-115', '146-190-137-170', '146-190-17-49', '146-190-224-102', '146-190-224-167', '146-190-226-100', '146-190-236-195', '146-190-253-190', '146-190-253-244', '146-190-43-174', '146-190-68-20', '147-182-156-252', '147-182-180-83', '147-182-212-94', '147-182-224-50', '152-42-139-14', '152-42-187-19', '152-42-200-67', '152-42-234-25', '152-42-243-208', '152-42-253-121', '157-230-104-19', '157-230-142-101', '157-230-148-111', '157-230-165-51', '157-230-169-154', '157-230-187-237', '157-230-189-190', '157-230-191-10', '157-230-37-84', '157-230-43-220', '157-245-101-137', '157-245-122-215', '157-245-129-49', '157-245-131-208', '157-245-138-157', '157-245-143-105', '157-245-149-131', '157-245-161-214', '157-245-164-121', '157-245-169-203', '157-245-177-238', '157-245-186-102', '157-245-187-113', '157-245-198-150', '157-245-211-183', '157-245-249-253', '157-245-252-13', '157-245-253-136', '157-245-38-8', '157-245-49-125', '157-245-49-231', '157-245-50-3', '157-245-54-20', '157-245-58-134', '157-245-63-134', '157-245-74-217', '157-245-94-47', '157-245-99-15', '159-203-10-171', '159-203-110-31', '159-203-112-193', '159-203-112-8', '159-203-13-12', '159-203-184-145', '159-203-220-240', '159-203-34-83', '159-203-38-15', '159-203-38-183', '159-203-42-172', '159-203-58-41', '159-203-65-155', '159-203-66-216', '159-203-8-155', '159-223-10-175', '159-223-148-164', '159-223-153-42', '159-223-154-174', '159-223-176-65', '159-223-181-112', '159-223-193-255', '159-223-194-190', '159-223-213-134', '159-223-4-2', '159-223-45-112', '159-223-5-143', '159-223-51-5', '159-223-69-154', '159-223-73-123', '159-65-103-74', '159-65-108-106', '159-65-13-116', '159-65-131-4', '159-65-145-122', '159-65-153-169', '159-65-154-187', '159-65-159-87', '159-65-17-108', '159-65-198-34', '159-65-242-27', '159-65-25-15', '159-65-35-215', '159-65-48-229', '159-65-52-30', '159-65-55-142', '159-65-65-232', '159-65-67-169', '159-65-67-43', '159-65-76-166', '159-65-87-42', '159-65-87-75', '159-65-91-109', '159-65-99-234', '159-89-113-242', '159-89-114-128', '159-89-115-83', '159-89-126-203', '159-89-127-103', '159-89-140-235', '159-89-149-247', '159-89-156-37', '159-89-164-112', '159-89-164-44', '159-89-165-194', '159-89-174-40', '159-89-193-30', '159-89-194-237', '159-89-21-191', '159-89-22-67', '161-35-125-121', '161-35-138-110', '161-35-150-103', '161-35-156-163', '161-35-164-161', '161-35-197-184', '161-35-198-209', '161-35-210-46', '161-35-213-51', '161-35-225-212', '161-35-235-116', '161-35-36-93', '161-35-39-134', '161-35-53-111', '161-35-53-32', '161-35-57-82', '161-35-73-249', '161-35-84-176', '162-243-103-117', '162-243-103-163', '162-243-104-223', '162-243-106-156', '162-243-118-215', '162-243-12-238', '162-243-128-251', '162-243-129-97', '162-243-130-183', '162-243-131-154', '162-243-131-166', '162-243-131-177', '162-243-134-172', '162-243-135-127', '162-243-135-69', '162-243-138-167', '162-243-139-160', '162-243-140-190', '162-243-140-228', '162-243-143-178', '162-243-146-230', '162-243-146-242', '162-243-151-124', '162-243-199-138', '162-243-199-139', '162-243-213-233', '162-243-215-55', '162-243-229-179', '162-243-231-144', '162-243-231-38', '162-243-235-44', '162-243-242-102', '162-243-250-78', '162-243-34-85', '162-243-4-131', '162-243-4-188', '162-243-42-197', '162-243-58-29', '162-243-77-96', '162-243-83-156', '162-243-98-7', '164-90-134-185', '164-90-141-111', '164-90-143-147', '164-90-153-178', '164-90-171-239', '164-90-195-222', '164-90-207-114', '164-90-207-134', '164-92-104-208', '164-92-109-106', '164-92-116-61', '164-92-118-182', '164-92-122-205', '164-92-130-186', '164-92-133-116', '164-92-141-156', '164-92-155-167', '164-92-167-30', '164-92-177-228', '164-92-201-214', '164-92-213-120', '164-92-222-189', '164-92-254-64', '164-92-71-87', '164-92-87-56', '164-92-88-17', '164-92-92-13', '164-92-95-128', '165-22-104-3', '165-22-114-111', '165-22-153-98', '165-22-154-134', '165-22-162-172', '165-22-170-132', '165-22-201-106', '165-22-218-164', '165-22-221-190', '165-22-222-115', '165-22-222-34', '165-22-233-156', '165-22-233-178', '165-22-37-5', '165-22-72-218', '165-22-85-194', '165-22-98-244', '165-227-123-161', '165-227-144-73', '165-227-18-3', '165-227-183-101', '165-227-200-97', '165-227-222-161', '165-227-28-139', '165-227-28-158', '165-227-36-148', '165-227-45-77', '165-227-56-150', '165-227-59-42', '165-227-89-124', '165-227-89-161', '165-227-92-32', '165-232-100-126', '165-232-108-247', '165-232-117-101', '165-232-136-236', '165-232-138-54', '165-232-158-89', '165-232-183-184', '165-232-185-55', '165-232-191-240', '165-232-53-137', '165-232-80-248', '167-172-106-230', '167-172-128-115', '167-172-136-226', '167-172-147-36', '167-172-148-16', '167-172-151-0', '167-172-151-249', '167-172-157-74', '167-172-164-4', '167-172-173-73', '167-172-174-162', '167-172-182-249', '167-172-183-108', '167-172-186-205', '167-172-191-248', '167-172-200-70', '167-172-201-99', '167-172-204-90', '167-172-205-222', '167-172-205-46', '167-172-225-125', '167-172-226-150', '167-172-233-31', '167-172-32-127', '167-172-42-88', '167-172-44-188', '167-172-76-0', '167-71-109-217', '167-71-123-210', '167-71-166-205', '167-71-22-138', '167-71-226-31', '167-71-231-43', '167-71-240-250', '167-71-7-172', '167-71-7-79', '167-99-108-141', '167-99-110-185', '167-99-118-78', '167-99-131-117', '167-99-176-66', '167-99-178-167', '167-99-189-223', '167-99-189-66', '167-99-207-47', '167-99-244-54', '167-99-254-158', '167-99-39-168', '167-99-50-1', '170-64-128-107', '170-64-134-250', '170-64-137-173', '170-64-138-253', '170-64-139-193', '170-64-141-105', '170-64-143-136', '170-64-144-66', '170-64-151-15', '170-64-153-87', '170-64-154-84', '170-64-157-109', '170-64-158-130', '170-64-159-2', '170-64-162-12', '170-64-166-50', '170-64-167-152', '170-64-175-206', '170-64-183-159', '170-64-193-147', '170-64-194-230', '170-64-195-19', '170-64-205-206', '170-64-205-224', '170-64-206-63', '170-64-215-121', '170-64-215-188', '170-64-222-64', '170-64-227-25', '170-64-229-67', '174-138-20-238', '174-138-24-131', '174-138-38-138', '174-138-50-253', '174-138-85-228', '174-138-87-9', '174-138-91-109', '174-138-92-234', '174-138-93-38', '178-128-107-227', '178-128-15-101', '178-128-157-196', '178-128-173-143', '178-128-178-58', '178-128-179-139', '178-128-181-15', '178-128-2-170', '178-128-2-41', '178-128-213-121', '178-128-231-147', '178-128-235-246', '178-128-238-222', '178-128-26-125', '178-128-3-254', '178-128-6-194', '178-128-74-241', '178-128-77-155', '178-128-77-89', '178-128-8-202', '178-128-80-199', '178-128-85-170', '178-128-93-60', '178-62-11-10', '178-62-144-97', '178-62-197-172', '178-62-204-165', '178-62-208-88', '178-62-226-224', '178-62-237-190', '178-62-241-234', '178-62-4-239', '178-62-50-75', '178-62-52-99', '178-62-56-174', '178-62-60-210', '178-62-73-26', '185-14-184-169', '185-14-184-186', '188-166-100-190', '188-166-120-38', '188-166-148-191', '188-166-153-80', '188-166-158-111', '188-166-170-1', '188-166-180-194', '188-166-188-134', '188-166-27-148', '188-166-3-149', '188-166-34-8', '188-166-69-247', '188-166-79-118', '188-166-83-42', '188-166-88-89', '188-166-94-154', '188-166-99-82', '188-226-133-149', '188-226-133-84', '188-226-136-244', '188-226-137-206', '188-226-141-57', '188-226-146-241', '188-226-148-69', '188-226-157-105', '188-226-157-125', '188-226-162-44', '188-226-173-190', '188-226-182-203', '188-226-248-237', '192-241-161-189', '192-241-165-164', '192-241-167-100', '192-241-177-132', '192-241-178-214', '192-241-178-40', '192-241-182-198', '192-241-182-22', '192-241-188-30', '192-241-197-127', '192-241-197-97', '192-241-200-224', '192-241-202-91', '192-241-205-80', '192-241-208-69', '192-241-210-117', '192-241-212-142', '192-241-212-148', '192-241-212-211', '192-241-213-37', '192-241-215-245', '192-241-223-63', '192-241-223-85', '192-241-225-93', '192-241-226-144', '192-241-226-154', '192-241-227-160', '192-241-228-172', '192-241-230-237', '192-241-230-241', '192-241-230-246', '192-241-231-109', '192-241-232-177', '192-241-233-149', '192-241-234-121', '192-241-239-192', '192-241-239-195', '192-241-246-112', '192-34-62-200', '192-81-219-206', '192-81-223-102', '198-199-100-163', '198-199-104-187', '198-199-106-194', '198-199-106-204', '198-199-107-45', '198-199-112-101', '198-199-112-172', '198-199-112-241', '198-199-113-218', '198-199-113-98', '198-199-116-106', '198-199-118-201', '198-199-118-219', '198-199-119-208', '198-199-126-34', '198-199-127-45', '198-199-65-51', '198-199-94-11', '198-199-94-193', '198-199-94-57', '198-199-96-53', '198-199-96-58', '198-211-121-174', '198-211-123-13', '198-211-127-223', '206-189-100-54', '206-189-104-15', '206-189-106-94', '206-189-117-202', '206-189-128-13', '206-189-14-136', '206-189-157-232', '206-189-158-26', '206-189-175-143', '206-189-18-129', '206-189-211-149', '206-189-211-252', '206-189-213-0', '206-189-225-10', '206-189-237-221', '206-189-30-92', '206-189-53-184', '206-189-67-95', '206-189-76-57', '206-189-97-247', '206-81-18-94', '206-81-9-214', '207-154-205-241', '207-154-205-25', '207-154-239-93', '209-38-103-65', '209-38-108-95', '209-38-120-221', '209-38-123-19', '209-38-123-57', '209-38-142-154', '209-38-151-16', '209-38-153-145', '209-38-166-186', '209-38-19-206', '209-38-209-72', '209-38-213-242', '209-38-22-23', '209-38-220-127', '209-38-226-233', '209-38-23-204', '209-38-24-8', '209-38-24-94', '209-38-27-59', '209-38-28-202', '209-38-30-114', '209-38-31-81', '209-38-33-189', '209-38-43-101', '209-38-47-117', '209-38-47-238', '209-38-82-124', '209-38-86-131', '209-38-86-175', '209-38-86-248', '209-38-87-163', '209-38-89-62', '209-38-90-236', '209-38-92-44', '209-38-93-31', '209-38-97-242', '209-97-128-179', '209-97-138-19', '209-97-156-224', '209-97-164-234', '209-97-166-253', '24-144-94-211', '24-199-112-72', '37-139-0-162', '37-139-0-94', '37-139-1-48', '37-139-10-251', '37-139-12-103', '37-139-12-162', '37-139-14-205', '37-139-14-215', '37-139-28-19', '37-139-28-90', '37-139-29-75', '37-139-3-239', '37-139-7-72', '37-139-9-69', '45-55-250-58', '45-55-75-44', '46-101-162-142', '46-101-174-218', '46-101-190-125', '46-101-248-70', '46-101-250-55', '46-101-38-158', '46-101-77-235', '46-101-80-95', '46-101-83-79', '46-101-94-213', '64-225-105-44', '64-225-117-196', '64-225-123-97', '64-225-127-141', '64-225-25-82', '64-225-29-112', '64-225-3-181', '64-225-5-182', '64-225-57-178', '64-225-65-219', '64-225-7-130', '64-225-73-146', '64-225-8-179', '64-226-119-139', '64-227-104-238', '64-227-106-59', '64-227-116-208', '64-227-123-196', '64-227-141-210', '64-227-145-243', '64-227-145-39', '64-227-148-144', '64-227-151-51', '64-227-154-254', '64-227-154-73', '64-227-160-196', '64-227-175-34', '64-227-190-154', '64-227-191-70', '64-227-7-228', '64-227-73-24', '64-227-81-233', '64-227-95-44', '64-23-134-39', '64-23-138-217', '64-23-139-191', '64-23-143-143', '64-23-146-214', '64-23-146-246', '64-23-149-250', '64-23-159-240', '64-23-163-233', '64-23-164-58', '64-23-167-216', '64-23-177-170', '64-23-179-121', '64-23-183-18', '64-23-210-96', '64-23-213-247', '67-205-134-92', '67-205-155-69', '67-205-184-177', '68-183-115-6', '68-183-163-188', '68-183-167-195', '68-183-176-92', '68-183-181-111', '68-183-181-13', '68-183-197-134', '68-183-207-218', '68-183-213-102', '68-183-22-3', '68-183-234-202', '68-183-4-102', '68-183-88-180', '68-183-88-252', '68-183-90-71', '68-183-91-86', '68-183-95-251', '80-240-132-104', '82-196-11-252', '95-85-10-85', '95-85-12-80', '95-85-15-184', '95-85-23-131', '95-85-23-220', '95-85-24-47', '95-85-25-105', '95-85-27-170', '95-85-30-187', '95-85-32-251', '95-85-33-143', '95-85-43-67', '95-85-6-182', '95-85-6-192', '95-85-61-125'])
    
    def convert_ip_format(ip):
        return ip.replace('-', '.')
    
    existing_ips = set(conn['id.resp_h'])
    
    ips = np.array([convert_ip_format(ip) for ip in ips])
    
    filtered_ip_array = np.array([ip for ip in ips if ip not in existing_ips])
    
    print(len(filtered_ip_array))
    print(filtered_ip_array)

    #csv: pd.DataFrame = pd.read_csv(f'{path_root}/Cloud-Telescope/data/raw/cloud_merged_pcap/zeek_v1/conn.log')
    #csv2: pd.DataFrame = csv[csv['ip'] == '152.32.237.160']
    #csv2
    print(conn[conn['id.orig_h'] == '104-131-10-181'])
    return conn
    
#test_mergecap()